# Hot song recommendations

Build song recommendation out of the k=100 nearest neighbors

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pandas as pd
import numpy as np
import matplotlib as plt
import importlib
from pyspark.ml.feature import Tokenizer, CountVectorizer, MinHashLSH
from pyspark.sql.types import IntegerType, StringType, ArrayType

import mpd

In [ ]:
# Will allow us to embed images in the notebook
%matplotlib inline
# change default plot size
plt.rcParams['figure.figsize'] = (15,10)

## Load and prep data

* Load the full data set
* Load the picked k=100 approx Nearest Neighbor results
* Build song recommdations based on songs in nearest playlist

In [ ]:
mpd_all=mpd.load(spark, "onebig", 1)

Get the ranked popularity of songs in the mpd.

In [ ]:
cv = CountVectorizer(inputCol="track_uri", outputCol="features", minDF=2, vocabSize=2000000)

In [ ]:
model=cv.fit(mpd_all.select("pid", "tracks.track_uri"))

In [ ]:
result=model.transform(mpd_all.select("pid", "tracks.track_uri"))

In [ ]:
#model, result = mpd.vectorizecol(mpd_all.select("pid", "tracks.track_uri"), "track_uri", "features", 2000000)

In [ ]:
result.printSchema()

In [ ]:
result.count()

In [ ]:
importlib.reload(mpd)

In [ ]:
vdf = mpd.buildvocabdf(spark, model.vocabulary)

In [ ]:
vdf.show(5)

In [ ]:
vdf.describe("tid").show()

In [ ]:
vdf.printSchema()

In [ ]:
vdf.count()

Get the Hot100 playlists that match the challenge set.

In [ ]:
hot100 = spark.createDataFrame(pd.read_pickle("ex-neighborpl.pkl"))

In [ ]:
hot100 = spark.createDataFrame(pd.read_pickle("neighborpl.pkl"))

In [ ]:
hot100.orderBy("pid").show(5)

In [ ]:
hot100.count()

In [ ]:
arraylength = f.udf(lambda x: len(x), IntegerType())

In [ ]:
h100cnt = hot100.withColumn("reclen", arraylength(hot100.recpl))

In [ ]:
h100cnt.orderBy("reclen").show()

In [ ]:
h100cnt.groupBy("reclen").count().orderBy("reclen").show(5)

In [ ]:
h100cnt.orderBy("reclen").groupBy("reclen").count().describe("count").show()

In [ ]:
h100cnt.describe("reclen").show()

We can see that most results will have gotten 100 neighbors

In [ ]:
mpd.plothist(h100cnt, "reclen", 11)

## Construct track list for neighboring playlists

We need to get all the tracks that are part of the MPD playlists in the neighborhood of the chellenge playlist.  We explode the playlist neighbors and then gather their track list for ranking and recommendation.

Make sure we still have all the entries resulting from the nearest neighbor search.

In [ ]:
hot100.groupBy("pid").count().count()

Create one row for each challenge playlist and each of its recommended playlists from the 100 nearest neighbors.  This will facilitate joining with the MPD playlists to get their track info.

We lose the 24 challenge playlists that didn't have nearest neighbors at this point because there are no MPD playlists to explode.

In [ ]:
h100 = hot100.select("pid", f.explode("recpl").alias("recpid"))

Keep a list of challenge playlists that don't have any identified nearest neighbors.  They dropped out during the explode process above and need to be brought back in later.

In [ ]:
missingh100pid = h100cnt.where(h100cnt.reclen == 0).select("pid")

In [ ]:
missingh100pid.show()

## The 24 zero length recpl entries go missing at this point.

In [ ]:
h100.groupBy("pid").count().count()

In [ ]:
h100withtracks = h100.join(result, result.pid == h100.recpid).drop(result.pid).drop(result.features).orderBy("pid")

In [ ]:
h100withtracks.show(5)

In [ ]:
h100withtracks.groupBy("pid").count().count()

## Elliminate duplicates with subtracting dataframes

It's possible to [use subtraction to remove the duplicates](https://stackoverflow.com/a/42380533)

Don't actually do this here though. Save it till after all the ordering takes place, otherwise we lose duplicates and the ability to count popularity.

challengetracks = recdf.select("pid", f.explode("tracks.track_uri").alias("track"))

challengetracks.count()

challengetracks.where(challengetracks.pid == 1000000).count()

tracklist = h100withtracks.select("pid", f.explode("track_uri").alias("track"))

tracklist.count()

newtracks = tracklist.subtract(challengetracks)

newtracks.count()

## Get the ranked resutls of tracks from the recommended neighboring playlists.

Make sure the elliminated tracks don't loose detail for a playlist.  The pid 1000000 order by count is different when the tracks aren't elliminated.   However the total count of tracks matches the expected difference.  646 original, less 5 given is 641.

In [ ]:
trackrank = h100withtracks.select("pid", f.explode("track_uri").alias("track")).groupBy("pid","track").count().sort(f.desc("count"))

In [ ]:
trackrank.orderBy("pid", f.desc("count")).show(5)

In [ ]:
trackrank.groupBy("pid").count().count()

In [ ]:
trackrank.where(trackrank.pid == 1000000).count()

Commented out for now favoring ellimination at the end

trackrank = newtracks.groupBy("pid","track").count().sort(f.desc("count"))

trackrank.orderBy("pid", f.desc("count")).show(5)

trackrank.where(trackrank.pid == 1000000).count()

In [ ]:
trackrank.where(trackrank.pid == 1000000).orderBy(f.asc("count")).show()

In [ ]:
trackrank.describe("count").show()

In [ ]:
trackrank.printSchema()

### Exlore a single playlist

In [ ]:
testpid = 1000061

In [ ]:
trackrank.where(f.col("pid") == testpid).show()

In [ ]:
trackrank.where(f.col("pid") == testpid).count()

## Add the global rank: another missing playlist makes 25

Without an outer join another playlist will go missing here.

In [ ]:
grank=trackrank.join(vdf, trackrank.track == vdf.term).drop(vdf.term)

In [ ]:
grank.printSchema()

In [ ]:
grank.groupBy("pid").count().count()

In [ ]:
grank.where(grank.tid.isNull()).count()

In [ ]:
gkrpid = grank.select("pid")

In [ ]:
tkrpid = trackrank.select("pid")

In [ ]:
missingpidgrank = tkrpid.subtract(gkrpid)

In [ ]:
missingpidgrank.show()

In [ ]:
grank.where(grank.pid.isNull()).count()

## Recommend for test playlist

Here is the track recommendation for one playlist based on the popularity of the track in the neighborhood with additional sorting by the globab popularity. Global popularity is based on count vecorizer with most popular recieving the lowest value.

In [ ]:
grank.where(f.col("pid") == testpid).orderBy(f.desc("count"), f.asc("tid")).show()

## Eliminate tracks included in the search

In [ ]:
mpd_test=spark.read.json("../mpd-challenge/challenge_set.json", multiLine=True)

In [ ]:
cpl=mpd_test.select(f.explode("playlists").alias("playlist"))

In [ ]:
recdf=cpl.select("playlist.pid", "playlist.tracks")

In [ ]:
recdf.printSchema()

In [ ]:
recdf.describe("pid").show()

In [ ]:
recdf.count()

## test a playlist

In [ ]:
existingtracks = recdf.where(recdf.pid == testpid).select(f.explode("tracks.track_uri").alias("track"))

In [ ]:
existingtracks.printSchema()

In [ ]:
existingtracks.show()

In [ ]:
existingtracks.toPandas()["track"].tolist()

In [ ]:
trank = grank.where(f.col("pid") == testpid).where(~grank.track.isin(existingtracks.toPandas()["track"].tolist()))

In [ ]:
trank.orderBy(f.desc("count"), f.asc("tid")).show()

## Iterate over search results and provide track list

In [ ]:
def gettracks(chpl, grank, recdf):
    # get the challenge playlist id
    testpid = chpl.pid
    
    # get the provided tracks
    existingtracks = recdf.where(recdf.pid == testpid).select(f.explode("tracks.track_uri").alias("track"))
    
    # get the tracks from the global rank
    df = grank.where(f.col("pid") == testpid).where(~grank.track.isin(existingtracks.toPandas()["track"].tolist()))
    
    tracklist = df.orderBy(f.desc("count"), f.asc("tid")).toPandas()["track"].tolist()
    
    recommend = { "pid": testpid, "tracks": [tracklist]}
    #print("DEBUG: " + testpid + " " + pidlist)
    
    return recommend


In [ ]:
recommended = pd.DataFrame({"pid":0, "tracks":[]})

Don't do this. It's awfully slow.

for row in hot100.limit(10).rdd.collect():
    rec = gettracks(row, grank, recdf)
    recommended = recommended.append(pd.DataFrame(rec))

In [ ]:
recommended

In [ ]:
recommended["tracks"].apply(lambda x: len(x))

recommended.to_pickle("rectracks.pkl")

recommended.to_csv("rectracks.csv")

## The iterrate-over-data-set is too damn slow 

The global ranking doesn't take much time even with a few cores.

In [ ]:
grank.orderBy("pid", f.desc("count"), f.asc("tid")).show()

Add monotonic id to preserve order.

In [ ]:
tgrank = grank.withColumn("mid", f.monotonically_increasing_id())

In [ ]:
tgrank = grank.orderBy("pid", f.desc("count"), f.asc("tid")).withColumn("mid", f.monotonically_increasing_id())

Now we have an mid that preserves the global order of our data set and can be used to sort tracks within playlists independent of nearest neighbor or global popularity counts.

In [ ]:
tgrank.orderBy("mid").show()

tgrank = tgrank.orderBy("pid", f.desc("count"), f.asc("tid")).withColumn("mid", f.monotonically_increasing_id()).drop("count").drop("tid")

In [ ]:
tgrank.count()

In [ ]:
tgrank = tgrank.drop("count").drop("tid")

In [ ]:
tgrank.printSchema()

In [ ]:
tgrank.groupBy("pid").count().count()

### Elliminate duplicates with subtracting dataframes

In [ ]:
challengetracks = recdf.select("pid", f.explode("tracks.track_uri").alias("track"))

In [ ]:
challengetracks.printSchema()

In [ ]:
challengetracks.count()

Impart monotonic id on challenge set to create identical rows

In [ ]:
ctracks = tgrank.join(challengetracks, [challengetracks.pid == tgrank.pid, challengetracks.track == tgrank.track]).drop(challengetracks.pid).drop(challengetracks.track)

In [ ]:
ctracks.printSchema()

In [ ]:
ctracks.count()

In [ ]:
ctracks.groupBy("pid").count().count()

In [ ]:
ctracks.orderBy("pid", "mid").show(truncate=False)

In [ ]:
newtracks = tgrank.subtract(ctracks)

Comparing to the above results it's clear we've ellimnated the provided tracks

In [ ]:
newtracks.count()

In [ ]:
newtracks.orderBy("pid", "mid").show()

Need to do the ellimination sooner because this approach loses the desired sorting.

Actually can do it here if we introduce a row index and do a join with the challenge set to get the duplicates with the same row id.  Then we can subtract and have identical rows and also retain the relative ordering.

Monotonically increasing should be fine since all we care about is sorting.

## Identify playlists that got skipped by the recommender

There are playlists that were completely empty sets because their elements didn't have any matches in the training set.

Do this by seeing which playlists are not in the recommended set and then create a data frame that can be unioned later.

In [ ]:
challengepids = recdf.select("pid")

In [ ]:
challengepids.count()

In [ ]:
recommendpids = hot100.select("pid")

In [ ]:
recommendpids.count()

In [ ]:
missingpids = challengepids.subtract(recommendpids)

In [ ]:
missingpids.count()

In [ ]:
missingpids.printSchema()

In [ ]:
missingpids.show()

In [ ]:
miss2 = missingpids.union(missingh100pid)

In [ ]:
miss3 = miss2.union(missingpidgrank)

In [ ]:
miss3.count()

In [ ]:
missingpids = miss3

In [ ]:
missingpids.show()

In [ ]:
arrayfill = f.udf(lambda x: [ ], ArrayType(StringType()))

In [ ]:
strfill = f.udf(lambda x: " ", StringType())

In [ ]:
missingpl = missingpids.withColumn("tracks", strfill(missingpids.pid))

mistest = missingpids.withColumn("tracks", arrayfill(f.lit("")))

mistest.show()

In [ ]:
missingpl.printSchema()

In [ ]:
missingpl.count()

In [ ]:
missingpl.show()

### Save data 

In [ ]:
trackrec = newtracks.orderBy("pid", "mid").groupBy("pid").agg(f.collect_list("track").alias("tracks"))

In [ ]:
trackrec.printSchema()

In [ ]:
trackrec.show(5)

In [ ]:
trackrec = trackrec.withColumn("len", arraylength(trackrec.tracks))

In [ ]:
trackrec.where(trackrec.len < 500).show()

In [ ]:
trackrec.count()

In [ ]:
trackrec.where(trackrec.len < 500).count()

## Add missing playlists and backfill from the top10000

In [ ]:
tolist = f.udf(lambda x: x.split(","), ArrayType(StringType()))

In [ ]:
missingpl2 = missingpl.withColumn("tlist", tolist("tracks"))

In [ ]:
missingpl2.printSchema()

In [ ]:
missingpl2.show()

In [ ]:
missingpl = missingpl2.drop("tracks").withColumnRenamed("tlist", "tracks")

In [ ]:
fullpl = missingpl.union(trackrec.select("pid", "tracks"))

In [ ]:
fullpl.count()

In [ ]:
fullpl.show()

## Backfill from the top10k

Choose from the Top 10000 songs as quick-n-dirty approach and with the potential for surprising variety to the recommendations

addlist = f.udf(lambda x: x + vdf.limit(10000).select("term").agg(f.collect_list("term".agg(f.collect_list("term")).alias("top10k"), ArrayType(StringType())))

def addlist(row, vdf, pdf):
    
    
    
    vdf.limit(10000).select("term").agg(f.collect_list("term".agg(f.collect_list("term")).alias("top10k")

fullpl.withColumn("list", top10k(fullpl))

## Get rid of all seed tracks from global vocab

In [ ]:
seedtracks = recdf.select(f.explode("tracks.track_uri").alias("track"))

In [ ]:
badtracks = vdf.join(seedtracks, seedtracks.track == vdf.term).drop(vdf.term)

In [ ]:
vdf.printSchema() 

In [ ]:
badtracks.printSchema()

In [ ]:
badtracks.count()

In [ ]:
goodtracks = vdf.subtract(badtracks)

In [ ]:
goodtracks.count()

In [ ]:
goodtracks.printSchema()

### Build backfill track list

This creates a string of tracks uris separated by commas so we can tack it on the end of the list of recommended tracks.

Go with 500 for now because driver memory error is getting hit on serialization

top10k = vdf.limit(600).select("term")

In [ ]:
top10k = goodtracks.orderBy("tid").limit(600).select("term")

In [ ]:
top10k.count()

In [ ]:
top10k.printSchema()

In [ ]:
top10kpd = top10k.toPandas()

In [ ]:
type(top10kpd)

In [ ]:
top10kstr = ', '.join(top10kpd["term"].tolist())

In [ ]:
type(top10kstr)

In [ ]:
mystr = str(top10kstr)

In [ ]:
type(mystr)

In [ ]:
top10kstr = mystr

joinstr = f.udf(lambda x: x.join(", "), StringType)

top10k = vdf.limit(10000).select("term").agg(f.collect_list("term").alias("top10k"))

top10kstr = top10k.select(f.format_string("%s", top10k.top10k).alias("top10kstr"))

top10kstr.printSchema()

type(top10kstr.top10kstr)

top10kstr.show()

top10k = vdf.limit(10000).select("term").agg(f.collect_list("term").alias("top10k")).rdd.collect()

type(top10k)

",".join(top10k.top10k)

top10k.printSchema()

top10k.show(truncate=True)

fullpl.withColumn("list", f.lit(fullp.pid))

top10k = vdf.limit(10000).select("term").agg(f.collect_list("term").alias("top10k")).toPandas()["top10k"].to_csv(index=False, header=False)

pd.set_option("display.max_colwidth", 1000)

top10kstr = top10k.toPandas().to_csv(index=False, header=False)

test = pd.read_csv(top10kstr)

f.regexp_replace(top10kstr, "[","")

top10k.printSchema

top10k.count()

type(top10k)

len(top10k)

In [ ]:
len(top10kstr)

pd.from_csv(top10kstr)

top10k

In [ ]:
putlit = f.udf(lambda x: x + top10kstr.split(","), ArrayType(StringType()))

In [ ]:
permute = f.udf(lambda x: np.random.permutation(x).tolist(), ArrayType(StringType()))

In [ ]:
tolist = f.udf(lambda x: x.split(","), ArrayType(StringType()))

In [ ]:
fullwithtop10k = fullpl.withColumn("top10kstr", f.lit(top10kstr))

In [ ]:
fullwithtop10k.printSchema()

In [ ]:
fullwithtop10k = fullwithtop10k.withColumn("top10k", putlit(fullwithtop10k.tracks))

Following is desired but not currently working. leads to null/udf errors later on

fullwithtop10k = fullwithtop10k.withColumn("top10k", tolist(fullwithtop10k.top10kstr)).drop("top10kstr")

In [ ]:
fullwithtop10k.printSchema()

fullwithtop10k.explain()

### Merge recommended tracks with filler tracks

In [ ]:
fullwithtop10k.select("pid").show(5)

fullwithtop10k.where(fullwithtop10k.tracks.isNull()).count()

fullwithtop10k.where(fullwithtop10k.top10kstr.isNull()).count()

fullwithtop10k.where(fullwithtop10k.pid.isNull()).count()

fullwithtop10k = fullwithtop10k.withColumn("top10k", tolist(fullwithtop10k.top10kstr)).drop(fullwithtop10k.top10kstr)

In [ ]:
fullwithtop10k.printSchema()

In [ ]:
fullwithtop10k.orderBy("pid").show()

In [ ]:
fullwithtop10k.printSchema()

No permutation for now because the construction of top10k is problematic

fullwithtop10kperm = fullwithtop10k.withColumn("permute", permute(fullwithtop10k.top10k))

fullwithtop10kperm.printSchema()

fullwithtop10kperm.count()

fullwithtop10k.describe("top10kstr")

makelist = f.udf(lambda x: x.split(","), ArrayType(StringType()))

readcsv = f.udf(lambda x: )

fullwithtop10k = fullwithtop10k.withColumn("top10k", makelist(fullwithtop10k.top10kstr))

In [ ]:
fullwithtop10k.count()

In [ ]:
fullwithtop10k.printSchema()

fullwithtop10k = fullwithtop10k.withColumn("tlen", arraylength(fullwithtop10k.top10k))

fullwithtop10k.describe("tlen")

fullwithpermute = fullwithtop10k.withColumn("permuted", permute(fullwithtop10k.top10k))

fullwithpermute.printSchema()

In [ ]:
mergeCols = f.udf((lambda x, y: x + y), ArrayType(StringType()))

fullcomplete = fullwithtop10kperm.withColumn("track10k", mergeCols(fullwithtop10kperm.tracks, fullwithtop10kperm.permute))

Accept simply top5000 priority ranking for now

In [ ]:
fullcomplete = fullwithtop10k

In [ ]:
fullcomplete.printSchema()

fullcomplete = fullcomplete.drop("tracks").drop("top10k").drop("permute")

fullcomplete = fullcomplete.withColumnRenamed("track10k", "tracks")

In [ ]:
fullcomplete = fullcomplete.drop("tracks").drop("top10kstr")

In [ ]:
fullcomplete = fullcomplete.withColumnRenamed("top10k", "tracks")

In [ ]:
fullcomplete.printSchema()

In [ ]:
fullcomplete = fullcomplete.withColumn("len", arraylength(fullcomplete.tracks))

In [ ]:
fullcomplete.describe("len").show()

## Limit all playlists to 500 tracks

If the first element is a blank space use 

In [ ]:
trunc500 = f.udf(lambda x: x[1:501] if x[0] == " " else x[0:500], ArrayType(StringType()))

In [ ]:
trunc500f = f.udf(lambda x: x[0:501], ArrayType(StringType()))

In [ ]:
cleanlist = fullcomplete.withColumn("first500", trunc500f(fullcomplete.tracks))

In [ ]:
cleanlist.printSchema()

In [ ]:
cleanlist.count()

## Format output

In [ ]:
trpd = cleanlist.select("pid","first500").toPandas()

trpd.to_csv("reclist.csv")

trpd.to_csv("reclist2.csv", quoting=0)

In [ ]:
trpd["formatted"] = trpd["pid"].map(str) + ", " + trpd["first500"].apply(', '.join)

trpd["formatted"]

type(trpd)

In [ ]:
trpd["formatted"].to_csv("recpl.csv", index=False, header=False)

In [ ]:
!echo -e "team_info,main,jprorama,jpr@uab.edu\n\n" > recpl-clean.csv

Fix the output becuase the quoting flag doesn't work above

In [ ]:
!sed -e 's/^"//' -e 's/"$//' recpl.csv >> recpl-clean.csv